In [23]:
import numpy as np
import pandas as pd
import datetime
import time

import torch
import torch.nn as nn
import matplotlib.pyplot as plt

为了防止未来函数，每一行中，所有在这一天内不应该得到的信息都必须标记前缀 'label_' <br>
例如，对于'20150304'这一天来说，close_ih是收盘时可以得到的，但是明天的收盘价 next_close 就必须标记为label_next_close<br>
在创建滞后变量的时候，尤其需要注意差分的方向。<br>

In [24]:
data = pd.read_csv('raw_market_state.csv', index_col='Unnamed: 0')
data.describe()

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,near_price_ic,far_price_ih,far_price_if,far_price_ic,near_maturity,far_maturity,on,m1,y1,y10
count,1.864000e+03,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1.864000e+03,1864.000000,1864.000000,1864.000000,...,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000
mean,2.021273e+07,2820.379269,2821.183517,2841.118077,2800.486885,-0.046317,6.584750e+07,4045.008781,4046.783074,4073.953404,...,5804.339914,2800.254399,3993.223927,5606.548605,18.374464,196.606760,1.867259,2.481915,2.591615,2.785440
std,2.335670e+04,355.761732,355.105308,358.570666,351.055944,34.217443,3.134920e+07,554.713031,554.125659,558.204697,...,731.086589,339.775288,529.612326,696.744399,9.396457,26.329254,0.506130,2.017771,1.825668,1.234936
min,2.017091e+07,2191.888700,2199.258700,2222.563100,2163.868200,-205.400100,1.705671e+07,2940.191600,2964.842100,3000.441300,...,3978.000000,2163.800000,2958.400000,3918.000000,3.000000,147.000000,0.441000,0.000000,0.040000,0.520000
25%,2.019091e+07,2553.599825,2559.550000,2573.870325,2535.919150,-16.099750,4.351735e+07,3718.195650,3720.195550,3745.301750,...,5266.050000,2559.850000,3666.250000,5089.800000,10.000000,176.000000,1.477000,0.180000,1.117500,1.680000
50%,2.021091e+07,2749.508250,2750.018850,2769.604650,2731.631550,0.075200,5.997245e+07,3933.547700,3935.427200,3959.101100,...,5889.600000,2727.400000,3894.500000,5726.900000,18.000000,197.000000,1.817500,2.100000,2.415000,2.850000
75%,2.023091e+07,2986.428100,2983.400500,3005.271600,2964.302650,16.489825,8.049420e+07,4228.312475,4237.830225,4269.055325,...,6312.300000,2981.350000,4178.250000,6087.950000,25.000000,219.000000,2.233000,4.420000,4.280000,3.990000
max,2.025092e+07,4097.866100,4028.528900,4110.179700,3996.628300,214.977900,3.322534e+08,5922.071400,5807.719100,5930.912200,...,7596.400000,3924.000000,5647.600000,7195.000000,37.000000,242.000000,3.282000,6.020000,5.490000,4.980000


In [25]:
data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,near_price_ic,far_price_ih,far_price_if,far_price_ic,near_maturity,far_maturity,on,m1,y1,y10
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,6639.4,2695.0,3797.0,6455.8,36,183,2.6810,0.99,1.28,2.20
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,6588.0,2698.0,3791.2,6419.2,35,182,2.6958,0.98,1.30,2.20
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,6678.4,2715.2,3834.4,6508.0,32,179,2.7560,0.96,1.30,2.23
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,6600.6,2702.8,3811.8,6472.0,31,178,2.8320,0.97,1.31,2.24
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,6668.6,2698.2,3826.0,6549.0,30,177,2.7900,0.98,1.32,2.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,7124.6,2991.2,4507.2,6823.2,8,190,1.3690,4.22,3.63,4.01
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,7140.0,2972.0,4478.0,6840.0,7,189,1.3670,4.24,3.66,4.06
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,7114.2,2964.0,4470.2,6780.2,4,186,1.4080,4.22,3.64,4.05
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,7165.2,2955.0,4466.0,6832.0,3,185,1.4370,4.20,3.62,4.04


In [26]:
# 接下来我们将从data创建两个表格
# feature
# 用于训练的数据，要求每个信息都在trade_date收盘时时可知信息
# 提前做好归一化处理

In [27]:
# 创建滞后变量-一定要注意检查方向！非常容易出现未来函数
for index in ['ih','if','ic']:
    data[f'%_diff_{index}_1'] = (data[f'close_{index}'].shift(1)-data[f'close_{index}'])/data[f'close_{index}']
    data[f'%_diff_{index}_5'] = (data[f'close_{index}'].shift(5)-data[f'close_{index}'])/data[f'close_{index}']
    data[f'%_diff_{index}_20'] = (data[f'close_{index}'].shift(20)-data[f'close_{index}'])/data[f'close_{index}']
    data[f'%_diff_{index}_60'] = (data[f'close_{index}'].shift(60)-data[f'close_{index}'])/data[f'close_{index}']

data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,%_diff_ih_20,%_diff_ih_60,%_diff_if_1,%_diff_if_5,%_diff_if_20,%_diff_if_60,%_diff_ic_1,%_diff_ic_5,%_diff_ic_20,%_diff_ic_60
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,NaN,NaN,-0.000350,NaN,NaN,NaN,0.007572,NaN,NaN,NaN
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,NaN,NaN,-0.003082,NaN,NaN,NaN,-0.007109,NaN,NaN,NaN
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,NaN,NaN,0.002877,NaN,NaN,NaN,0.005038,NaN,NaN,NaN
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,NaN,NaN,-0.002686,NaN,NaN,NaN,-0.008435,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,-0.057022,-0.099921,-0.022574,-0.040199,-0.081674,-0.148249,-0.026759,-0.059564,-0.086288,-0.190222
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,-0.046848,-0.095868,0.005758,-0.013638,-0.077109,-0.144100,-0.003504,-0.032709,-0.100438,-0.195423
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,-0.043792,-0.095420,-0.002439,-0.014446,-0.072954,-0.145175,0.001455,-0.020350,-0.079692,-0.194959
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,-0.036959,-0.092985,0.002149,-0.019251,-0.062769,-0.149600,-0.007458,-0.036438,-0.072705,-0.215753


In [28]:
# 创建升贴水率变量
for index in ['ih','if','ic']:
    data[f'near_discount_{index}'] = (data[f'near_price_{index}']-data[f'close_{index}'])/data[f'close_{index}']/data['near_maturity'] * 250
    data[f'far_discount_{index}'] = (data[f'far_price_{index}']-data[f'close_{index}'])/data[f'close_{index}']/data['far_maturity'] * 250
    data[f'amount_{index}'] = data[f'amount_{index}']

data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,%_diff_ic_1,%_diff_ic_5,%_diff_ic_20,%_diff_ic_60,near_discount_ih,far_discount_ih,near_discount_if,far_discount_if,near_discount_ic,far_discount_ic
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,NaN,NaN,NaN,NaN,0.017573,0.011328,-0.019864,-0.011755,-0.034123,-0.044304
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,0.007572,NaN,NaN,NaN,0.033793,0.015250,-0.015468,-0.014376,-0.036721,-0.042076
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,-0.007109,NaN,NaN,NaN,0.042724,0.021426,0.023697,-0.003177,0.010478,-0.033810
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,0.005038,NaN,NaN,NaN,0.019654,0.016886,-0.017501,-0.007446,-0.043045,-0.034714
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,-0.008435,NaN,NaN,NaN,0.039351,0.018870,0.006430,-0.006041,-0.029725,-0.030279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,-0.026759,-0.059564,-0.086288,-0.190222,0.074557,0.003580,0.095958,-0.011814,0.008314,-0.055328
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,-0.003504,-0.032709,-0.100438,-0.195423,0.000725,0.001542,0.009493,-0.012870,-0.038718,-0.056952
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,0.001455,-0.020350,-0.079692,-0.194959,-0.004544,0.000628,-0.072464,-0.018637,-0.202830,-0.067260
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,-0.007458,-0.036438,-0.072705,-0.215753,0.078527,0.003290,-0.120425,-0.017129,-0.298902,-0.067463


In [29]:
# 创建label变量 作为y 注意这里都是未来函数，不可用于feature，只可用于label
# 绝对收益
for index in ['ih','if','ic']:
    data[f'label_{index}_ch_next_day'] = data[f'close_{index}'].shift(-1)-data[f'close_{index}']
    data[f'label_{index}_ch_next_week'] = data[f'close_{index}'].shift(-5)-data[f'close_{index}']
    data[f'label_{index}_ch_next_hfmonth'] = data[f'close_{index}'].shift(-10)-data[f'close_{index}']
    data[f'label_{index}_ch_next_month'] = data[f'close_{index}'].shift(-20)-data[f'close_{index}']
    data[f'label_{index}_ch_next_hfyear'] = data[f'close_{index}'].shift(-100)-data[f'close_{index}']

# 方向符号（用于训练）
for index in ['ih','if','ic']:
    data[f'label_{index}_sign_next_day'] = (data[f'label_{index}_ch_next_day'] > 0).astype(float)
    data[f'label_{index}_sign_next_week'] = (data[f'label_{index}_ch_next_week'] > 0).astype(float)
    data[f'label_{index}_sign_next_hfmonth'] = (data[f'label_{index}_ch_next_hfmonth'] > 0).astype(float)
    data[f'label_{index}_sign_next_month'] = (data[f'label_{index}_ch_next_month'] > 0).astype(float)
    data[f'label_{index}_sign_next_hfyear'] = (data[f'label_{index}_ch_next_hfyear'] > 0).astype(float)

# 实现波动率（辅助训练）
    data[f'label_volatility_{index}_week'] = data[f'amount_{index}'].rolling(window = 5).mean().shift(-4)
    data[f'label_volatility_{index}_hfmonth'] = data[f'amount_{index}'].rolling(window = 10).mean().shift(-9)

data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,label_if_sign_next_hfyear,label_volatility_if_week,label_volatility_if_hfmonth,label_ic_sign_next_day,label_ic_sign_next_week,label_ic_sign_next_hfmonth,label_ic_sign_next_month,label_ic_sign_next_hfyear,label_volatility_ic_week,label_volatility_ic_hfmonth
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,1.0,1.460000e+08,1.348810e+08,0.0,0.0,0.0,0.0,0.0,1.268434e+08,1.133340e+08
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,1.0,1.462043e+08,1.321345e+08,1.0,0.0,0.0,0.0,0.0,1.219868e+08,1.082333e+08
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,1.0,1.411810e+08,1.283924e+08,0.0,0.0,1.0,0.0,0.0,1.158262e+08,1.040025e+08
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,1.0,1.378089e+08,1.289063e+08,1.0,0.0,1.0,1.0,0.0,1.131798e+08,1.040834e+08
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,1.0,1.289104e+08,1.287687e+08,0.0,0.0,0.0,0.0,0.0,1.061810e+08,1.044236e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,0.0,6.436460e+08,NaN,1.0,0.0,0.0,0.0,0.0,4.582893e+08,NaN
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN


In [30]:
data = data.copy()

In [31]:
# 创建一些技术指标
for index in ['ih','if','ic']:

    # MA
    for delta in [5, 10, 20, 60]:
        data[f'MA_{index}_{delta}'] = data[f'close_{index}'].rolling(window = delta).mean()

    # MACD
    data[f'EMA_short_{index}'] = data[f'close_{index}'].ewm(span=10, adjust=False).mean()
    data[f'EMA_long_{index}'] = data[f'close_{index}'].ewm(span=20, adjust=False).mean()
    data[f'DIF_{index}'] = data[f'EMA_short_{index}'] - data[f'EMA_long_{index}']
    data[f'DEA_{index}'] = data[f'DIF_{index}'].ewm(span=10, adjust=False).mean()
    data[f'MACD_{index}'] = (data[f'DIF_{index}'] - data[f'DEA_{index}']) * 2
    data[f'long_signals_{index}'] = ((data[f'DIF_{index}'] > data[f'DEA_{index}']) & (data[f'DIF_{index}'].shift(1) < data[f'DEA_{index}'].shift(1))).astype(float)
    data[f'short_signals_{index}'] = ((data[f'DIF_{index}'] < data[f'DEA_{index}']) & (data[f'DIF_{index}'].shift(1) > data[f'DEA_{index}'].shift(1))).astype(float)

    # RSI 参数为10
    data[f'RSI_up_{index}'] = data[f'change_{index}'].where(data[f'change_{index}'] > 0, 0)
    data[f'RSI_down_{index}'] = data[f'change_{index}'].where(data[f'change_{index}'] < 0, 0)
    data[f'RSI_ave_up_{index}'] = data[f'RSI_up_{index}'].ewm(alpha = 1/10, min_periods = 10).mean()
    data[f'RSI_ave_down_{index}'] = data[f'RSI_down_{index}'].ewm(alpha = 1/10, min_periods = 10).mean()
    data[f'RS_{index}'] = data[f'RSI_ave_up_{index}'] / data[f'RSI_ave_down_{index}']
    data[f'RSI_{index}'] = 100 - (100/ (1 + data[f'RS_{index}']))

    # ATR
    data[f'amplitude_{index}'] = (data[f'high_{index}'] - data[f'low_{index}']) / data[f'close_{index}'].shift(1)
    data[f'amplitude_up_{index}'] = (data[f'high_{index}'] - data[f'close_{index}'].shift(1)) / data[f'close_{index}'].shift(1)
    data[f'amplitude_low_{index}'] = (data[f'close_{index}'].shift(1) - data[f'low_{index}']) / data[f'close_{index}'].shift(1)
    data[f'amplitude_avg_{index}'] = data[f'amplitude_{index}'] + data[f'amplitude_up_{index}'] + data[f'amplitude_low_{index}']
    data[f'ATR_{index}'] = data[f'amplitude_avg_{index}'].ewm(span = 10, min_periods = 10).mean()



In [32]:
from scipy.stats import norm
def down_probability(row):
    return norm.cdf(-45, loc = row['label_ih_ch_next_week'], scale=row['alter_std'])

def middle_probability(row):
    return norm.cdf(45, loc = row['label_ih_ch_next_week'], scale=row['alter_std']) - norm.cdf(-45, loc = row['label_ih_ch_next_week'], scale=row['alter_std'])

def up_probability(row):
    return 1 - norm.cdf(45, loc = row['label_ih_ch_next_week'], scale=row['alter_std'])

In [33]:
data['alter_std'] = data['label_volatility_ih_week']/data['label_volatility_ih_week'].mean()*180
data['down_probability'] = data.apply(down_probability, axis = 1)
data['middle_probability'] = data.apply(middle_probability, axis = 1)
data['up_probability'] = data.apply(up_probability, axis = 1)
data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,RSI_ic,amplitude_ic,amplitude_up_ic,amplitude_low_ic,amplitude_avg_ic,ATR_ic,alter_std,down_probability,middle_probability,up_probability
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,NaN,NaN,NaN,NaN,NaN,NaN,106.511406,0.327086,0.327232,0.345682
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,NaN,0.008960,-0.001300,0.010260,0.017920,NaN,108.159834,0.303358,0.321147,0.375494
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,NaN,0.008053,0.007160,0.000893,0.016106,NaN,102.812068,0.308299,0.337761,0.353940
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,NaN,0.010421,0.001769,0.008652,0.020842,NaN,100.327455,0.297620,0.345125,0.357256
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,NaN,0.011913,0.008543,0.003370,0.023826,NaN,92.999264,0.318345,0.371502,0.310153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,215.788280,0.033679,0.027545,0.006134,0.067358,0.047593,449.933338,NaN,NaN,NaN
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,205.684240,0.012318,0.010874,0.001444,0.024635,0.043419,NaN,NaN,NaN,NaN
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,214.301509,0.006724,0.003797,0.002927,0.013448,0.037970,NaN,NaN,NaN,NaN
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,192.151230,0.016112,0.008135,0.007977,0.032224,0.036925,NaN,NaN,NaN,NaN


In [34]:
data.to_csv('market_state.csv')

In [35]:
data

,trade_date,open_ih,close_ih,high_ih,low_ih,change_ih,amount_ih,open_if,close_if,high_if,...,RSI_ic,amplitude_ic,amplitude_up_ic,amplitude_low_ic,amplitude_avg_ic,ATR_ic,alter_std,down_probability,middle_probability,up_probability
0,20170914,2688.3998,2672.8364,2694.7750,2666.8169,-14.8343,3.652983e+07,3843.5720,3829.9554,3854.4931,...,NaN,NaN,NaN,NaN,NaN,NaN,106.511406,0.327086,0.327232,0.345682
1,20170915,2669.6530,2668.3759,2677.5329,2659.7328,-4.4605,4.324333e+07,3826.1448,3831.2965,3843.2970,...,NaN,0.008960,-0.001300,0.010260,0.017920,NaN,108.159834,0.303358,0.321147,0.375494
2,20170918,2670.1977,2674.1759,2694.0589,2670.1977,5.8000,4.447027e+07,3832.9626,3843.1428,3862.3589,...,NaN,0.008053,0.007160,0.000893,0.016106,NaN,102.812068,0.308299,0.337761,0.353940
3,20170919,2680.7159,2670.6914,2687.1837,2661.6095,-3.4845,3.748888e+07,3849.8794,3832.1161,3857.3030,...,NaN,0.010421,0.001769,0.008652,0.020842,NaN,100.327455,0.297620,0.345125,0.357256
4,20170920,2667.2401,2662.6266,2675.2112,2661.8681,-8.0648,3.288359e+07,3830.1549,3842.4353,3850.9426,...,NaN,0.011913,0.008543,0.003370,0.023826,NaN,92.999264,0.318345,0.371502,0.310153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20250911,2925.6584,2983.0829,2983.1968,2917.8084,43.4918,1.884955e+08,4438.8390,4548.0345,4548.0345,...,215.788280,0.033679,0.027545,0.006134,0.067358,0.047593,449.933338,NaN,NaN,NaN
1860,20250912,2986.0229,2968.5397,2998.9807,2964.5848,-14.5432,1.774533e+08,4544.5905,4521.9980,4564.6665,...,205.684240,0.012318,0.010874,0.001444,0.024635,0.043419,NaN,NaN,NaN,NaN
1861,20250915,2976.3452,2962.6154,2984.8555,2957.7164,-5.9243,1.456983e+08,4534.8866,4533.0557,4566.9053,...,214.301509,0.006724,0.003797,0.002927,0.013448,0.037970,NaN,NaN,NaN,NaN
1862,20250916,2969.2337,2947.8222,2974.9485,2942.2965,-14.7932,1.554296e+08,4539.9217,4523.3367,4552.4190,...,192.151230,0.016112,0.008135,0.007977,0.032224,0.036925,NaN,NaN,NaN,NaN
